In [7]:
using CausalForest
using RCall
using StatsBase
using BenchmarkTools

In [8]:
indices,X,T,Y = load_data("causal")
reval("set.seed(1)")
Xtest_1 = rcopy(R"matrix(runif(10000),nrow=1000)")
true_effect_1 = Xtest_1[:, 1].>0.5;

# Critère exact : centré honnête

### Subsampling

In [10]:
errors_7 = zeros(100)

cf = @btime build_forest(true, false, true, Y, T, X, true, 10, 500, 0.5, 0.5, -1, 5, 10, 2000)
pred = @btime apply_forest(cf, Xtest_1)
errors_7[1] = rmsd(float(true_effect_1), pred)
for i in 2:100
    @rput i
    reval("set.seed(i)")
    Xtest = rcopy(R"matrix(runif(10000),nrow=1000)")
    pred = apply_forest(cf, Xtest)
    true_effect = Xtest[:, 1].>0.5
    errors_7[i] = rmsd(float(true_effect), pred)
end
print(mean(errors_7))
print(var(errors_7))

  157.057 s (47559709 allocations: 6.03 GiB)
  11.721 s (139309992 allocations: 2.31 GiB)
0.092194758207655452.6772147781223152e-5

### Bootstrap

In [11]:
errors_8 = zeros(100)

cf = @btime build_forest(true, true, true, Y, T, X, true, 10, 500, 0.5, 0.5, -1, 5, 10, 2000)
pred = @btime apply_forest(cf, Xtest_1)
errors_8[1] = rmsd(float(true_effect_1), pred)
for i in 2:100
    @rput i
    reval("set.seed(i)")
    Xtest = rcopy(R"matrix(runif(10000),nrow=1000)")
    pred = apply_forest(cf, Xtest)
    true_effect = Xtest[:, 1].>0.5
    errors_8[i] = rmsd(float(true_effect), pred)
end
print(mean(errors_8))
print(var(errors_8))

  191.654 s (48696357 allocations: 6.26 GiB)
  12.871 s (151128883 allocations: 2.49 GiB)
0.094367957879971382.3688574876047828e-5

# Critère alternatif (différence non centrée) honnête

### Subsampling

In [12]:
errors_11 = zeros(100)

cf = @btime build_forest_1(false, false, true, Y, T, X, true, 10, 500, 0.5, 0.5, -1, 5, 10, 2000)
pred = @btime apply_forest_1(cf, Xtest_1)
errors_11[1] = rmsd(float(true_effect_1), pred)
for i in 2:100
    @rput i
    reval("set.seed(i)")
    Xtest = rcopy(R"matrix(runif(10000),nrow=1000)")
    pred = apply_forest_1(cf, Xtest)
    true_effect = Xtest[:, 1].>0.5
    errors_11[i] = rmsd(float(true_effect), pred)
end
print(mean(errors_11))
print(var(errors_11))

  14.265 s (1568787 allocations: 405.13 MiB)
  11.841 s (132298315 allocations: 2.21 GiB)
0.098892343428435753.0791265443823704e-5

### Bootstrap

In [14]:
errors_12 = zeros(100)

cf = @btime build_forest_1(false, true, true, Y, T, X, true, 10, 500, 0.5, 0.5, -1, 5, 10, 2000)
pred = @btime apply_forest_1(cf, Xtest_1)
errors_12[1] = rmsd(float(true_effect_1), pred)
for i in 2:100
    @rput i
    reval("set.seed(i)")
    Xtest = rcopy(R"matrix(runif(10000),nrow=1000)")
    pred = apply_forest_1(cf, Xtest)
    true_effect = Xtest[:, 1].>0.5
    errors_12[i] = rmsd(float(true_effect), pred)
end
print(mean(errors_12))
print(var(errors_12))

  41.349 s (2963560 allocations: 669.71 MiB)
  10.648 s (137272161 allocations: 2.28 GiB)
0.125170037001155455.246458417280409e-5